In [ ]:
import requests
import os
from bs4 import BeautifulSoup

In [ ]:
def parsing(url):
     #요청 헤더 설정 : 브라우저 정보
    req_header = {
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36"
    }

    # requests 의 get() 함수 호출하기 
    res=requests.get(url,headers=req_header)
    res.encoding = 'utf-8'

    # 응답(response)이 OK 이면 text 추출
    if res.ok:
        html = res.text
        # url에서 파일명만 추출하기
        file_name = os.path.basename(url)
        # BeautifulSoup 객체 생성
        soup = BeautifulSoup(html, "html.parser")  
        result_set = soup.select('ul.list_newsheadline2 li')
        print(type(result_set),len(result_set))

In [ ]:
from urllib.parse import urlparse
import os

def parse_filename_from_direct_url(url):
    """
    URL이 직접적인 파일 경로일 때 파일 이름을 파싱합니다.
    """
    parsed_url = urlparse(url)
    # URL 경로에서 마지막 부분을 가져옵니다.
    path = parsed_url.path
    # os.path.basename은 경로의 마지막 구성 요소를 반환합니다.
    filename = os.path.basename(path)
    return filename

# 예시
url1 = "http://example.com/documents/report.pdf"
url2 = "https://www.google.com/images/branding/googlelogo/1x/googlelogo_color_272x92dp.png"
url3 = "http://example.com/index.html" # HTML 파일도 결국은 파일이름이 있습니다.

print(f"URL: {url1} -> File Name: {parse_filename_from_direct_url(url1)}")
print(f"URL: {url2} -> File Name: {parse_filename_from_direct_url(url2)}")
print(f"URL: {url3} -> File Name: {parse_filename_from_direct_url(url3)}")



In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import os

def parse_filenames_from_html_page(url):
    """
    웹페이지의 HTML을 파싱하여 파일 링크(PDF, 이미지, 등)를 찾고 파일 이름을 추출합니다.
    """
    try:
        response = requests.get(url)
        response.raise_for_status() # HTTP 오류 발생 시 예외 발생

        soup = BeautifulSoup(response.text, 'html.parser')

        found_filenames = set() # 중복 제거를 위해 set 사용

        # 1. <a> 태그의 href 속성에서 파일 확장자를 가진 링크 찾기
        # 일반적인 문서/미디어 파일 확장자를 추가할 수 있습니다.
        file_extensions = ('.pdf', '.doc', '.docx', '.xls', '.xlsx', '.ppt', '.pptx',
                           '.zip', '.rar', '.tar', '.gz', '.7z', 'exe',
                           '.jpg', '.jpeg', '.png', '.gif', '.svg', '.mp4', '.mp3')

        for a_tag in soup.find_all('a', href=True):
            href = a_tag['href']
            # 절대 URL로 변환하여 상대 경로도 처리 가능
            abs_url = urljoin(url, href)
            if abs_url.lower().endswith(file_extensions):
                filename = os.path.basename(urlparse(abs_url).path)
                if filename: # 빈 문자열이 아닌 경우에만 추가
                    found_filenames.add(filename)

        # 2. <img> 태그의 src 속성에서 파일 이름 찾기
        for img_tag in soup.find_all('img', src=True):
            src = img_tag['src']
            abs_url = urljoin(url, src)
            filename = os.path.basename(urlparse(abs_url).path)
            if filename:
                found_filenames.add(filename)

        # 3. <source> 태그 (video/audio)
        for source_tag in soup.find_all('source', src=True):
            src = source_tag['src']
            abs_url = urljoin(url, src)
            filename = os.path.basename(urlparse(abs_url).path)
            if filename:
                found_filenames.add(filename)
        #4. <div> 태그 
        for source_tag in soup.find_all('div', src=True):
            src = source_tag['src']
            abs_url = urljoin(url, src)
            filename = os.path.basename(urlparse(abs_url).path)
            if filename:
                found_filenames.add(filename)

        return list(found_filenames) # set을 list로 변환하여 반환

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL {url}: {e}")
        return []
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return []


current_url = "https://math-son.tistory.com/1387"
filenames = parse_filenames_from_html_page(current_url)
print(f"\nFiles found on {current_url}: {filenames}")


# # 단순 설명을 위한 URL 예시
# print("\n--- HTML 페이지 파싱 예시 ---")
# # 실제 요청은 하지 않고, HTML 문자열을 직접 파싱하여 시연
# html_content_for_test = """
# <html>
# <body>
#     <a href="http://example.com/documents/report.pdf">Download PDF</a>
#     <img src="/images/logo.png">
#     <a href="/data/archive.zip">Archive File</a>
#     <a href="javascript:void(0);">No File Here</a>
#     <a href="https://other.com/photos/image.jpg">External Image</a>
# </body>
# </html>
# """
# soup_test = BeautifulSoup(html_content_for_test, 'html.parser')
# test_url_base = "http://example.com/" # 상대 경로 처리를 위한 기본 URL

# found_filenames_manual = set()
# file_extensions = ('.pdf', '.doc', '.docx', '.xls', '.xlsx', '.ppt', '.pptx',
#                    '.zip', '.rar', '.tar', '.gz', '.7z',
#                    '.jpg', '.jpeg', '.png', '.gif', '.svg', '.mp4', '.mp3')

# for tag in soup_test.find_all(['a', 'img', 'source']):
#     src_or_href = tag.get('href') or tag.get('src')
#     if src_or_href:
#         abs_url = urljoin(test_url_base, src_or_href)
#         if abs_url.lower().endswith(file_extensions):
#             filename = os.path.basename(urlparse(abs_url).path)
#             if filename:
#                 found_filenames_manual.add(filename)

# print(f"Manually parsed filenames: {list(found_filenames_manual)}")



Files found on https://math-son.tistory.com/1387: ['2026%20%EC%88%98%EB%8A%A5%ED%8A%B9%EA%B0%95%20%EC%98%81%EC%96%B4%EB%8F%85%ED%95%B4%EC%97%B0%EC%8A%B5.pdf', '2026_%EC%88%98%EB%8A%A5%ED%8A%B9%EA%B0%95_%EC%96%B8%EC%96%B4%EC%99%80_%EB%A7%A4%EC%B2%B4_%EC%A0%95%EB%8B%B5%EA%B3%BC%ED%95%B4%EC%84%A4.pdf', '2026_%EC%88%98%EB%8A%A5%ED%8A%B9%EA%B0%95_%EC%98%81%EC%96%B4_%EC%A0%95%EB%8B%B5%EA%B3%BC%ED%95%B4%EC%84%A4.pdf', '9946E6465CDBC72506', '2026%20%EC%88%98%EB%8A%A5%ED%8A%B9%EA%B0%95%20%EB%AF%B8%EC%A0%81%EB%B6%84.pdf.pdf', '2026_%EC%88%98%EB%8A%A5%ED%8A%B9%EA%B0%95_%EB%AC%B8%ED%95%99_%EC%A0%95%EB%8B%B5%EA%B3%BC%ED%95%B4%EC%84%A4.pdf', '2026_%EC%88%98%EB%8A%A5%ED%8A%B9%EA%B0%95_%EC%84%B8%EA%B3%84%EC%82%AC_%EC%A0%95%EB%8B%B5%EA%B3%BC%ED%95%B4%EC%84%A4%20(2).pdf', '2026_%EC%88%98%EB%8A%A5%ED%8A%B9%EA%B0%95_%EC%82%AC%ED%9A%8C%EB%AC%B8%ED%99%94_%EC%A0%95%EB%8B%B5%EA%B3%BC%ED%95%B4%EC%84%A4.pdf', '2026_%EC%88%98%EB%8A%A5%ED%8A%B9%EA%B0%95_%EC%98%81%EC%96%B4%EB%8F%85%ED%95%B4%EC%97%B0%EC%8A%B5_%EB%